In [1]:
import pandas as pd
import janitor
import numpy as np
from IPython.display import display
from utilities import pandas_to_tex

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", None)

import sys

sys.path.append("/home/lsys/pwned_pols//venv/lib/python3.10/site-packages")

In [2]:
df = (
    pd.read_csv(
        "../data/everypol/everypol_combined_legislature_data.csv", low_memory=False
    )
    .dropna(subset=["email"])
    .query("email!='-'")
    .query("email!='No tiene'")
    # ================================================================
    # Fix missing cc3 for Wales/Scotland
    # Wales, Scotland = GBR
    .assign(
        cc3=lambda df_: np.where(
            df_["cc"] == "GB-SCT",
            "GBR",
            np.where(df_["cc"] == "GB-WLS", "GBR", df_["cc3"]),
        )
    )
    # ================================================================
    # Fix missing pop for Wales/Scotland
    .assign(
        pop2024=lambda df_: np.where(
            df_["cc"] == "GB-SCT",
            "GBR",
            np.where(df_["cc"] == "GB-WLS", 68556800, df_["pop2024"]),
        )
    )
    # ================================================================
    # Keep only unique email by earliest known date
    .sort_values(["cc3", "leg_start_year", "email"])
    .drop_duplicates(subset=["email"], keep="first", ignore_index=True)
    # ================================================================
    # Get #emails per country (cc3)
    .assign(nemail_cc3=lambda df_: df_.groupby("cc3")["email"].transform("nunique"))
    # ================================================================
    # Perc female
    .assign(
        female_count_cc3=lambda df_: df_.groupby("cc3")["gender"].transform(
            lambda g: (g == "female").sum()
        ),
        female_prop_cc3=lambda df_: df_["female_count_cc3"] / df_["nemail_cc3"],
    )
    # ================================================================
    # Get indicator for years and chambers
    .assign(
        years=lambda df_: df_.groupby("cc3")["leg_start_year"].transform(
            lambda x: ", ".join(map(str, sorted(x.unique())))
        ),
        chambers=lambda df_: df_.groupby("cc3")["ltype"]
        .transform(lambda x: ", ".join(map(str, sorted(x.unique()))))
        .replace("legislature", "", regex=True)
        .replace("house", "", regex=True)
        .str.strip()
        .str.title(),
        legislatures=lambda df_: df_.groupby("cc3")["legislature"].transform(
            lambda x: ", ".join(map(str, sorted(x.unique())))
        ),
    )
    # ================================================================
    .reorder_columns(
        [
            "email",
            "cc3",
            "country",
            "ltype",
            "legislature",
            "chamber",
            "leg_start_year",
            "nemail_cc3",
            "gender",
        ]
    )
)

assert (df["female_count_cc3"] <= df["nemail_cc3"]).all()
assert (df["female_prop_cc3"] <= 1).all()
display(df.head(3))
df.info()

,email,cc3,country,ltype,legislature,chamber,leg_start_year,nemail_cc3,gender,id,name,sort_name,twitter,facebook,group,group_id,area_id,area,term_x,start_date,end_date,image,wikidata,wikidata_group,wikidata_area,row_id,term_y,row_count,n_unique_emails,person_count_legistype,url,cc,leg_start_date,lastmod,pop2024,lastmod_year,female_count_cc3,female_prop_cc3,years,chambers,legislatures
0,albana.vokshi@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,female,c8bd71fc-4815-40b5-a5c4-bd359e3b0cef,Albana Vokshi,VOKSHI ALBANA,NaN,NaN,PD,c26946f7-8c5e-4474-a85e-ee5342cf4006,durrës_county,Durrës County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Albana-Vokshi-PD.jpg,Q4709025,Q845743,NaN,490,VII Pluralist Legislature,140,31,292,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1557734928,2740502.0,2019,12,0.085714,"2009, 2013, 2017",Unicameral,Kuvendi
1,aldo.bumci@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,male,589ba883-b893-40ed-95f8-69b10f0497d9,Aldo Bumçi,BUMÇI ALDO,NaN,NaN,PD,c26946f7-8c5e-4474-a85e-ee5342cf4006,tirana_county,Tirana County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Aldo-Bumci-PD.jpg,Q2832310,Q845743,NaN,490,VII Pluralist Legislature,140,31,292,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1557734928,2740502.0,2019,12,0.085714,"2009, 2013, 2017",Unicameral,Kuvendi
2,bashkim.fino@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,male,9d0181c7-7ebe-4b16-800a-6ea28baf22f5,Bashkim Fino,FINO BASHKIM,NaN,NaN,PS,0c83d4f1-9e86-48f2-95ee-9c3d158d1141,fier_county,Fier County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Bashkim-Fino-PS.jpg,Q809978,Q642882,NaN,490,VII Pluralist Legislature,140,31,292,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1557734928,2740502.0,2019,12,0.085714,"2009, 2013, 2017",Unicameral,Kuvendi


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8536 entries, 0 to 8535
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   email                   8536 non-null   object 
 1   cc3                     8536 non-null   object 
 2   country                 8536 non-null   object 
 3   ltype                   8536 non-null   object 
 4   legislature             8536 non-null   object 
 5   chamber                 8536 non-null   object 
 6   leg_start_year          8536 non-null   int64  
 7   nemail_cc3              8536 non-null   int64  
 8   gender                  7320 non-null   object 
 9   id                      8536 non-null   object 
 10  name                    8536 non-null   object 
 11  sort_name               8536 non-null   object 
 12  twitter                 2398 non-null   object 
 13  facebook                1602 non-null   object 
 14  group                   8535 non-null   

In [3]:
# countries with incomplete gender coverage
for c in df["cc3"].unique().tolist():
    _df = df.query(f"cc3=='{c}'")
    n_gender = len(_df.dropna(subset=["gender"]))
    n_country = len(_df)
    if n_gender < n_country:
        name = _df.reset_index().loc[0, "country"]
        print(f"{name} ({c}):\n {n_gender}, {n_country}")

Albania (ALB):
 61, 140
Armenia (ARM):
 32, 119
Bulgaria (BGR):
 188, 208
Cameroon (CMR):
 83, 111
Colombia (COL):
 55, 169
Estonia (EST):
 95, 101
Georgia (GEO):
 50, 146
Guernsey (GGY):
 17, 40
Guatemala (GTM):
 143, 152
Hungary (HUN):
 171, 184
Iran (IRN):
 51, 139
Italy (ITA):
 284, 314
South-Korea (KOR):
 238, 253
Moldova (MDA):
 43, 44
Macedonia (MKD):
 95, 101
Namibia (NAM):
 12, 72
Nigeria (NGA):
 107, 140
Nicaragua (NIC):
 2, 85
Nepal (NPL):
 47, 269
Papua-New-Guinea (PNG):
 59, 60
Rwanda (RWA):
 1, 77
Slovakia (SVK):
 117, 165
Seychelles (SYC):
 0, 32
Tanzania (TZA):
 384, 405
Uruguay (URY):
 108, 119
South-Africa (ZAF):
 371, 385


In [4]:
set_edomain = set()
for _, row in df.iterrows():
    try:
        edomain = row["email"].split("@")[1]
        set_edomain.add(edomain)
    except IndexError:
        print(_)
#         print(row["email"])
        
len(set_edomain)

504

## Basic numbers

In [5]:
df["nemail_cc3"].describe()

count    8536.000000
mean      284.852390
std       221.909397
min        31.000000
25%       140.000000
50%       209.000000
75%       405.000000
max       835.000000
Name: nemail_cc3, dtype: float64

In [6]:
df["ltype"].unique()

array(['unicameral legislature', 'lower house', 'upper house'],
      dtype=object)

In [7]:
df["legislature"].unique()

array(['Kuvendi', 'Consell General', 'National Assembly',
       'House of Representatives', 'Senate', 'Chamber of Representatives',
       'Parliament', 'House of Commons', 'Assemblée Nationale',
       'Cámara de Representantes', 'Folketing', 'Riigikogu', 'Eduskunta',
       'Scottish Parliament', 'National Assembly for Wales',
       'Parliament of Georgia', 'States', 'Hellenic Parliament',
       'Inatsisartut', 'Congress', 'Legislative Council', 'Országgyűlés',
       'Lok Sabha', 'Majles', 'Chamber of Deputies', 'Parlament',
       'Sobranie', 'National Council', 'Tweede Kamer',
       'Constituent Assembly', 'New Zealand Parliament',
       'National Parliament', 'Assembly', 'House of Assembly'],
      dtype=object)

In [8]:
df["legislature"].nunique()

34

In [9]:
df["id"].nunique()

8536

In [10]:
df["email"].nunique()

8536

In [11]:
df["cc3"].nunique()

55

In [12]:
df.groupby(["cc3", "legislature"]).ngroups

61

In [13]:
df["female_prop_cc3"].describe()

count    8536.000000
mean        0.227858
std         0.127633
min         0.000000
25%         0.117647
50%         0.250000
75%         0.337725
max         0.543860
Name: female_prop_cc3, dtype: float64

## Table1

In [14]:
df_tab1 = (
    df.drop_duplicates(["cc3"], ignore_index=True)
    #     .filter(["cc3", "country", "nemail_cc3", "female_prop_cc3", "years", "chambers", "legislatures", "pop2024"])
    #     .assign(female_prop_cc3=lambda df_: (100 * df_["female_prop_cc3"]).round(1).astype(str) + "\\%")
    .filter(
        ["cc3", "country", "nemail_cc3", "years", "chambers", "legislatures", "pop2024"]
    )
    .assign(pop2024=lambda df_: (df_["pop2024"].astype(float) / 1_000_000).round(1))
    .replace("Bosnia-and-Herzegovina", "Bosnia")
    .replace(
        "House of Commons, National Assembly for Wales, Scottish Parliament",
        "Commons, Senedd, Scottish Parliament",
    )
    .replace("1997, 2001, 2005, 2007, 2010, 2011, 2015, 2016, 2017", "1997--2017")
    .replace("2004, 2007, 2010, 2013, 2016", "2004--2016")
    .replace("2001, 2005, 2007, 2011, 2015", "2001--2015")
    .replace("2004, 2007, 2009, 2012, 2015", "2004--2015")
    .replace("Lower ", "Lower", regex=True)
    .assign(ix=lambda df_: range(1, 1 + len(df_)))
    .reorder_columns(["ix"])
)
df_tab1

,ix,cc3,country,nemail_cc3,years,chambers,legislatures,pop2024
0,1,ALB,Albania,140,"2009, 2013, 2017",Unicameral,Kuvendi,2.7
1,2,AND,Andorra,31,2015,Unicameral,Consell General,0.1
2,3,ARM,Armenia,119,2019,Unicameral,National Assembly,2.8
3,4,AUS,Australia,179,2004--2016,"Lower, Upper","House of Representatives, Senate",26.9
4,5,BEL,Belgium,149,2014,Lower,Chamber of Representatives,11.9
5,6,BGR,Bulgaria,208,"2013, 2014, 2017",Unicameral,National Assembly,6.4
6,7,BIH,Bosnia,42,2014,Lower,House of Representatives,3.2
7,8,BLR,Belarus,60,2016,Unicameral,House of Representatives,9.1
8,9,BMU,Bermuda,33,2017,Lower,Parliament,NaN
9,10,BTN,Bhutan,48,2013,Lower,National Assembly,0.8


In [15]:
df_tab1["pop2024"].astype(float).sum()

2608.4

In [16]:
# % coverage of global pop
100 * (df_tab1["pop2024"].astype(float).sum()) / 8000

32.605

In [17]:
pandas_to_tex(df_tab1, "test.tex", index=False, na_rep="---")

/home/lsys/pwned_pols/scripts/utilities.py:204: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tex_table = df.to_latex(index=index, header=False, escape=escape, **kwargs)


In [18]:
!cat test.tex

 1 & ALB &          Albania & 140 &       2009, 2013, 2017 &         Unicameral &                              Kuvendi &    2.7 \\
 2 & AND &          Andorra &  31 &                   2015 &         Unicameral &                      Consell General &    0.1 \\
 3 & ARM &          Armenia & 119 &                   2019 &         Unicameral &                    National Assembly &    2.8 \\
 4 & AUS &        Australia & 179 &             2004--2016 &       Lower, Upper &     House of Representatives, Senate &   26.9 \\
 5 & BEL &          Belgium & 149 &                   2014 &              Lower &           Chamber of Representatives &   11.9 \\
 6 & BGR &         Bulgaria & 208 &       2013, 2014, 2017 &         Unicameral &                    National Assembly &    6.4 \\
 7 & BIH &           Bosnia &  42 &                   2014 &              Lower &             House of Representatives &    3.2 \\
 8 & BLR &          Belarus &  60 &                   2016 &         Unicameral &  

## Merge to HIBP

In [19]:
df_email_breach_expanded = (
    # All EP emails
    df
    .filter(['email', "gender", 'cc3', 'country', 'ltype', 'legislature', 'chamber',
       'leg_start_year', 'nemail_cc3'])
    # ============================================================================
    # Merge to breached status
    .merge(
        (
            pd.read_csv("../data/everypol/everypol_hibp.csv")
            .clean_names()
            .rename_column("filename", "email")
        ), how="left", on="email", validate="1:m"
    )
    .dropna(subset=["breach"])
    # ============================================================================
    # Merge to expanded breach info
    .merge(
        (
            pd.read_parquet("../data/breaches_01_2025_expanded.parquet")
            .remove_columns(["description", "title", "domain", "breachdate", "addeddate", "modifieddate", "logopath", "timetopublic", "dataclasses"])
            .rename_column("name", "breach")
        ), how="left", on="breach", validate="m:1"
    )
)
df_email_breach_expanded

,email,gender,cc3,country,ltype,legislature,chamber,leg_start_year,nemail_cc3,breach,present,pwncount,isverified,isfabricated,issensitive,isretired,isspamlist,ismalware,issubscriptionfree,yearstopublic,n_dataclasses,Places of birth,Car ownership statuses,Drinking habits,Security questions and answers,Loyalty program details,Home ownership statuses,Mothers maiden names,Instant messenger identities,Mnemonic phrases,Partial credit card data,Eating habits,Relationship statuses,Customer interactions,Clothing sizes,Vehicle identification numbers (VINs),Deceased statuses,Address book contacts,Drug habits,Credit card CVV,Age groups,Loan information,Physical attributes,IMSI numbers,Usernames,Social connections,HIV statuses,Time zones,Job titles,Email messages,...,Family structure,Account balances,Credit cards,Spoken languages,Career levels,Avatars,Encrypted keys,Apps installed on devices,Device information,Political donations,Phone numbers,Nationalities,Driver's licenses,Bios,Ethnicities,Credit status information,Geographic locations,Photos,Email addresses,Website activity,Dates of birth,Charitable donations,Living costs,Payment methods,Beauty ratings,Sexual orientations,Private messages,Password hints,Sexual fetishes,Smoking habits,Personal descriptions,Browser user agent details,Bank account numbers,Reward program balances,Citizenship statuses,Profile photos,Cryptocurrency wallet addresses,Passwords,Purchasing habits,Browsing histories,Deceased date,Years of professional experience,Buying preferences,Licence plates,Chat logs,User website URLs,Financial investments,Password strengths,Spouses names,Support tickets
0,edmond.spaho@parlament.al,male,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,000webhost,False,14936670,True,False,False,False,False,False,False,0.654795,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,edmond.spaho@parlament.al,male,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,123RF,False,8661578,True,False,False,False,False,False,False,0.652055,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,edmond.spaho@parlament.al,male,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,2844Breaches,False,80115532,False,False,False,False,False,False,False,0.019178,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,edmond.spaho@parlament.al,male,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,500px,False,14867999,True,False,False,False,False,False,False,0.720548,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,edmond.spaho@parlament.al,male,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,8fit,False,15025407,True,False,False,False,False,False,False,0.720548,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660967,wmaondera@gmail.com,male,ZWE,Zimbabwe,lower house,House of Assembly,House of Assembly,2013,37,digiDirect,False,304337,True,False,False,False,False,False,False,0.071233,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Manually check

In [20]:
non_breached_emails = []
for pol in df["email"].unique():
    if pol not in df_email_breach_expanded["email"].unique().tolist():
        non_breached_emails.append(pol)

In [22]:
import random
random.seed(42)

# Sample 10 unique emails
sampled_emails = random.sample(non_breached_emails, 20)
sampled_emails

['ogroba@parlamento.gub.uy',
 'azizabdel@assnat.cm',
 'harutyun.babayan@parliament.am',
 'anupam.hazra@sansad.nic.in',
 'ermontufar@congreso.gob.gt',
 'g.akriotis@parliament.gr',
 'modesto.aguilera@camara.gov.co',
 'Tony.Dean@sen.parl.gc.ca',
 'pmwdaka@parliament.gov.zm',
 'lubomir_vazny@nrsr.sk',
 'kwilson@parliament.bm',
 'c.ruge@bunge.go.tz',
 'i.stoilkovik@sobranie.mk',
 'vahe.hovhannisyan1@parliament.am',
 'sipan.pashinyan@parliament.am',
 'uwangdi@nab.gov.bt',
 'g.germenis@parliament.gr',
 'malo@inatsisartut.gl',
 'gilda.vaiho-faatoa@assemblee.pf',
 'm.haji@bunge.go.tz']

Paste (no breach) 

* g.akriotis@parliament.gr

### Basic numbers

In [23]:
# 2964 pols have been breached
n_pwned_pols = df_email_breach_expanded["email"].nunique()
n_pwned_pols

2964

In [24]:
100*n_pwned_pols/len(df)

34.72352389878163

In [25]:
# 1173 pols have been breached multiple times
n_multi_pwned_pols = len(
    df_email_breach_expanded
    .groupby(["email"])["present"]
    .sum()
    .reset_index()
    .query("present==1")
)
n_multi_pwned_pols

1173

In [26]:
# Proportion of pawned
100 * (n_multi_pwned_pols/n_pwned_pols)

39.5748987854251

In [27]:
# Proportion of total
100 * (n_multi_pwned_pols/len(df))

13.741799437675725

### How many email-pw pairs

In [29]:
(
    df_email_breach_expanded
    .clean_names()
    .query("email_addresses==1")
    .query("passwords==1")
    ["email"]
    .nunique()
)

2964